In [1]:
# to run GUI event loop
%autosave 180
%matplotlib qt
import matplotlib.pyplot as plt
plt.ion()

import numpy as np
#import nidaqmx
#from nidaqmx import stream_writers
import time
from tqdm import trange
from pypylon import pylon
#from imageio import get_writer
import cv2

Autosaving every 180 seconds


In [3]:
############################################
############INITIALIZE VIDEO ###############
############################################

#
try:
    grab.Release()
    camera.Close()
except:
    pass


fname_video = r'C:\code\test_data\test.avi'
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
fps = 30
video_width = 1900
video_height = 1200
frame_exposure_time = 10000

# this is the single_channel_version 
video_single_channel_flag = True
video_out = cv2.VideoWriter(fname_video,
                             fourcc,
                             fps,
                             (video_width, video_height))


#
tl_factory = pylon.TlFactory.GetInstance()
devices = tl_factory.EnumerateDevices()
for device in devices:
    print(device.GetFriendlyName())

tl_factory = pylon.TlFactory.GetInstance()
camera = pylon.InstantCamera()
camera.Attach(tl_factory.CreateDevice(devices[0]))
print("DeviceClass: ", camera.GetDeviceInfo().GetDeviceClass())
print("DeviceFactory: ", camera.GetDeviceInfo().GetDeviceFactory())
print("ModelName: ", camera.GetDeviceInfo().GetModelName())

# 
camera.Open()
camera.ExposureTime.SetValue(20000)  # exposure time in microseconds
if False:
    camera.TriggerSource.SetValue("Line4")
    camera.TriggerMode.SetValue("On")
    camera.TriggerActivation.SetValue('RisingEdge')
    camera.TriggerSelector.SetValue('FrameStart')

if True:    
    #camera.TriggerSelector.SetValue(FrameStart);
    #camera.TriggerMode.SetValue(TriggerMode_On);
    #camera.TriggerSource.SetValue(TriggerSource_Software);
    camera.TriggerSoftware.Execute();

# 
camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)

# grab a video frame
grab = camera.RetrieveResult(10000, 
                         pylon.TimeoutHandling_ThrowException)
#writer.append_data()
frame = grab.GetArray()
gray = cv2.normalize(frame, None, 255, 0, 
                     norm_type=cv2.NORM_MINMAX, 
                     dtype=cv2.CV_8U)
gray_3c = cv2.merge([gray, gray, gray])


subsample_rate = 1

# camera_obj = ax.imshow(gray_3c[::subsample_rate,
#                                ::subsample_rate,
#                                ::subsample_rate],
#                               vmin=0,
#                               vmax=255,
#                               aspect='auto',
#                               cmap='binary_r',
#                       animated=True)

# #
# plt.show(block=False)
# plt.pause(0.2)
# fig, ax = plt.subplots()
# bg = fig.canvas.copy_from_bbox(fig.bbox)
# # draw the animated artist, this uses a cached renderer
# ax.draw_artist(camera_obj)
# # show the result to the screen, this pushes the updated RGBA buffer from the
# # renderer to the GUI framework so you can see it
# fig.canvas.blit(fig.bbox)


#
n_frames = 300
for k in trange(n_frames):
    grab = camera.RetrieveResult(1000, 
                             pylon.TimeoutHandling_ThrowException)
    #writer.append_data()
    frame = grab.GetArray()

    #
    gray = cv2.normalize(frame, None, 255, 0, 
                         norm_type=cv2.NORM_MINMAX, 
                         dtype=cv2.CV_8U)

    #
    gray_3c = cv2.merge([gray, gray, gray])
    if True:
        video_out.write(gray_3c)
    
    camera.ExecuteSoftwareTrigger

#     #######################################
#     #######################################
#     if True:
#         fig.canvas.restore_region(bg)
#         camera_obj.set_data(gray_3c[::subsample_rate,
#                                ::subsample_rate,
#                                ::subsample_rate])
#         ax.draw_artist(camera_obj)
        
#         #
#         fig.canvas.blit(fig.bbox)
        
#         # 
#         fig.canvas.flush_events()

try:
    camera.Close()
except:
    print ("WARNING CAMERA didn't close")

plt.close()

print ("Done")


Basler acA2040-55um (23490633)
DeviceClass:  BaslerUsb
DeviceFactory:  USB/BaslerUsb 6.3.0.18933
ModelName:  acA2040-55um


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:06<00:00, 47.42it/s]

Done


In [6]:

# 
fname_video = r'C:\code\test_data\test.avi'
fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
fps = 30
video_width = 1900
video_height = 1200
frame_exposure_time = 10000

# this is the single_channel_version 
video_single_channel_flag = True
video_out = cv2.VideoWriter(fname_video,
                             fourcc,
                             fps,
                             (video_width, video_height))




# TODO: is this required anymore?
try:
    camera.Close()
except:
    pass

#
tl_factory = pylon.TlFactory.GetInstance()
devices = tl_factory.EnumerateDevices()
for ctr,device in enumerate(devices):
    print("vid camera: ", ctr, device.GetFriendlyName())

camera_id = 0
print ("Selecting video camera ID # ", camera_id)

tl_factory = pylon.TlFactory.GetInstance()
camera = pylon.InstantCamera()
camera.Attach(tl_factory.CreateDevice(devices[camera_id]))
print("DeviceClass: ", camera.GetDeviceInfo().GetDeviceClass())
print("DeviceFactory: ", camera.GetDeviceInfo().GetDeviceFactory())
print("ModelName: ", camera.GetDeviceInfo().GetModelName())

# 
camera.Open()
camera.Width = video_width
camera.Height = video_height
camera.ExposureTime.SetValue(frame_exposure_time)  # exposure time in microseconds

# # put camera in hardware trigger mode if selected
# if hardware_trigger_flag:
#     self.camera.TriggerSource.SetValue("Line4")
#     self.camera.TriggerMode.SetValue("On")
#     self.camera.TriggerActivation.SetValue('RisingEdge')
#     self.camera.TriggerSelector.SetValue('FrameStart')

####################################################
# start the camera to wait for triggers
camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly)
# grab a video frame
grab = camera.RetrieveResult(10000, 
                         pylon.TimeoutHandling_ThrowException)
#writer.append_data()
frame = grab.GetArray()
gray = cv2.normalize(frame, None, 255, 0, 
                     norm_type=cv2.NORM_MINMAX, 
                     dtype=cv2.CV_8U)
gray_3c = cv2.merge([gray, gray, gray])
#################################


fig, ax = plt.subplots()

subsample_rate = 1

camera_obj = ax.imshow(gray_3c[::subsample_rate,
                               ::subsample_rate,
                               ::subsample_rate],
                              vmin=0,
                              vmax=255,
                              aspect='auto',
                              cmap='binary_r',
                      animated=True)

#
plt.show(block=False)
plt.pause(0.2)

bg = fig.canvas.copy_from_bbox(fig.bbox)
# draw the animated artist, this uses a cached renderer
ax.draw_artist(camera_obj)
# show the result to the screen, this pushes the updated RGBA buffer from the
# renderer to the GUI framework so you can see it
fig.canvas.blit(fig.bbox)




######################################################################
######################################################################
######################################################################
video_frame_times = []
for k in range(10):
    
    start_time = time.time()
    #
    grab = camera.RetrieveResult(10000, 
                                   pylon.TimeoutHandling_ThrowException)

    # get the image data from array
    frame = grab.GetArray()

    # format the image to be saved for
    if video_single_channel_flag == False:
        gray = cv2.normalize(frame, None, 255, 0,
                                 norm_type=cv2.NORM_MINMAX,
                                 dtype=cv2.CV_8U)
        gray_3c = cv2.merge([gray, gray, gray])
        video_out.write(gray_3c)
    else:
        #gray = cv2.cvtColor(np.float32(imgUMat), cv2.COLOR_RGB2GRAY)
        gray = cv2.normalize(frame, None, 255, 0,
                                 norm_type=cv2.NORM_MINMAX,
                                 dtype=cv2.CV_8U)
        video_out.write(frame)

    #
    video_frame_times.append([k,start_time])
    
    #
    time.sleep(0.033)

    if True:
        fig.canvas.restore_region(bg)
        camera_obj.set_data(gray_3c[::subsample_rate,
                               ::subsample_rate,
                               ::subsample_rate])
        ax.draw_artist(camera_obj)

        #
        fig.canvas.blit(fig.bbox)

        # 
        fig.canvas.flush_events()
        
        #

plt.close()

#
        
np.save(r'C:\code\test_data\video_frame_times.npy',
        video_frame_times)
try:
    # close out the video writer
    video_out.release()
except:
    print ("WARNING VIDEO WRITER DIDN'T CLOSE")
try:
    camera.Close()
except:
    print ("WARNING CAMERA didn't close")
    
print ("DONE VIDEO...")

vid camera:  0 Basler acA2040-55um (23490633)
Selecting video camera ID #  0
DeviceClass:  BaslerUsb
DeviceFactory:  USB/BaslerUsb 6.3.0.18933
ModelName:  acA2040-55um
DONE VIDEO...
